The proposal for this project is to use machine learning to detect patterns in the sensor data so that the stackeholer will be better to exploit the forecasting and act accordingly by maintaining the unterlying system. 

This notebook is keept in such a way as I went trough it. However, weak results are not interfering with final outcomes and predictions.

We are going to use the following python libraries and therefore the upcoming cell is going to install it for us.

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.holtwinters import SimpleExpSmoothing

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_squared_error

from statsmodels.tsa.stattools import adfuller


# Data Loading

In [ ]:
# np.genfromtxt('sensor.csv', delimiter=',', dtype=None)
df = pd.read_csv('sensor.csv')

# Exploratory Data Analysis

In [ ]:
df.info()

There are some missing values. Some columns will be dropped as seen in the upcoming column. 
Those columns with only a few missing values will be 'backfilled' with the pandas method in the later stage of this notebook.

In [ ]:
df.isnull().sum().plot(kind='bar', figsize=(12,1));
plt.title('Number of Missing Values in a Column');

In [ ]:
df.drop(['Unnamed: 0','sensor_00','sensor_15','sensor_50','sensor_51'], axis=1, inplace=True)

In the next step visual inspectation of each sensor record is made. 

In [ ]:
# Transforming mashine status from strings to integers 
conditions = [(df['machine_status'] =='NORMAL'), (df['machine_status'] =='BROKEN'), (df['machine_status'] =='RECOVERING')]
choices = [1, 0, 0.5]
df['Operation'] = np.select(conditions, choices, default=0)

We can see group by some sensors in terms of their behavior and the absolute numbers they display. All sensors show stationary behavior, which means that their values are not changing from the initial value in time. Non-stationary behaviour we be that the values are rising / or falling in time.

In [ ]:
# time consuming step!

# for i in df.drop(['timestamp', 'machine_status', 'Operation'], axis = 1).dropna().T.to_numpy():
#    result = adfuller(i)
#    print(result[1])

In [ ]:
# commented out because time expensive
# df.set_index('timestamp').plot(subplots =True, sharex = True, figsize = (20,50));

In [ ]:
# Statistics of the machine. It is a highly imbalanced data set
df.machine_status.value_counts()

In [ ]:
# Status of the machine. 1 = operational, 0.5 maintenance and 0 = broken
df.set_index('timestamp').Operation.plot(figsize=(13,1));
plt.ylabel('Machine Status');

### Time Lag

In [ ]:
# https://www.kaggle.com/code/ryanholbrook/time-series-as-features
# With trend and seasonality, we trained models to fit curves to plots like those on the left in the figure above -- 
# the models were learning time dependence. The goal in this lesson is to train models to fit curves to plots like those on the right 
# we want them to learn serial dependence

In [ ]:
df['Lag_1'] = df['sensor_01'].shift(1)
#df = df.reindex(columns=['Hardcover', 'Lag_1'])

In [ ]:
sns.regplot(x='Lag_1', y='sensor_01', data=df, ci=None, scatter_kws=dict(color='0.25'));

In [ ]:
df['timestamp'] = pd.to_datetime(df.timestamp)

In [ ]:
df = df.backfill() # somehow fillna('backfill') produced columns with objects.')
# df.info()

In [ ]:
m = 6000
df.sensor_23.plot(figsize = (12, 2))
df.rolling(window=m).mean()['sensor_23'].plot()

## Machnine Learning

In [ ]:
# The absolute sensor values scatter a lot. For ML it is better to keep all values in a certain range eg. between 0 - 1.
mms = MinMaxScaler()

In [ ]:
to_convert = ['sensor_01', 'sensor_02', 'sensor_03', 'sensor_04',
       'sensor_05', 'sensor_06', 'sensor_07', 'sensor_08', 'sensor_09',
       'sensor_10', 'sensor_11', 'sensor_12', 'sensor_13', 'sensor_14',
       'sensor_16', 'sensor_17', 'sensor_18', 'sensor_19', 'sensor_20',
       'sensor_21', 'sensor_22', 'sensor_23', 'sensor_24', 'sensor_25',
       'sensor_26', 'sensor_27', 'sensor_28', 'sensor_29', 'sensor_30',
       'sensor_31', 'sensor_32', 'sensor_33', 'sensor_34', 'sensor_35',
       'sensor_36', 'sensor_37', 'sensor_38', 'sensor_39', 'sensor_40',
       'sensor_41', 'sensor_42', 'sensor_43', 'sensor_44', 'sensor_45',
       'sensor_46', 'sensor_47', 'sensor_48', 'sensor_49']

In [ ]:
df_scaled = pd.DataFrame(mms.fit_transform(df[to_convert]))

In [ ]:
df = pd.concat([df, df_scaled], axis = 1).drop(to_convert, axis= 1)

In [ ]:
df = df.backfill() # somehow fillna('backfill') produced columns with objects.')

In [ ]:
df.isna().describe()

In [ ]:
df.set_index('timestamp', inplace=True)
df.index.freq = 'min' # df.index gets the frequency of the time series. It is needed for future steps.

In [ ]:
# train/test split time series
train_df = df.loc[df.index < "2018-06-09 10:40:00"]
test_df = df.loc[df.index >= "2018-06-09 10:40:00"]
X_train = train_df.drop(['machine_status', 'Operation'], axis = 1)
y_train = train_df.Operation
X_test = test_df.drop(['machine_status', 'Operation'], axis = 1)
y_test = test_df.Operation

In [ ]:
# A good overview how the traces look now
# train_df.plot(subplots = True, sharex = True, figsize= (12,30));
# X_train.info()

### Linear Regression

In [ ]:
reg = LinearRegression()
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)

In [ ]:
y_test_plot = y_test.copy()
y_test_plot = pd.DataFrame(y_test_plot)
y_test_plot['y_pred'] = y_pred.tolist()
y_test_plot.plot.line(figsize=(15,2));
plt.title('Linear Regression Prediction');

In [ ]:
print('RMSE for Linear Regression: ', "%.3f" % mean_squared_error(y_pred, y_test)**(1/2))

### Random Forrest

too long to run therfore commented out

In [ ]:
# rf = RandomForestRegressor()
# rf.fit(X_train, y_train)
# y_pred = rf.predict(X_test)

In [ ]:
# y_test_plot = pd.DataFrame(y_test_plot)
# y_test_plot['y_pred'] = y_pred.tolist()
# y_test_plot.plot.line(figsize=(15,2));
# plt.title('Random Forest Prediction');

In [ ]:
# print('RMSE for Random Forest: ', "%.3f" % mean_squared_error(y_test.Operation, y_pred)**(1/2))

## Convert a Time Series to a Supervised Learning Problem: Sliding Window

Conclusively, random forest show a lower root mean square error (RMSE) than linear regression algorithm. It can better predict the underlying machine status. However, those example are not predicitions in advance. As both algorothms predict the machine fallout within a minute, which is to short
to take action or do a maintanace precedure. Therefore, a shift in the window function is necessary.

Shifting the  features 60 steps to be able to see the failure of the machine 60 minutes in advance.

You can play around with the time shift. Positive values push the time trace more into the future, while negative values pull the time back.
1440 minutes corresponds to a single day. The amount of time before the machine stops working.

In [ ]:
for i in df.columns:
    if i == 'timestamp':
        continue
    else:
        for t in [180, 1440, 2880, 5760]: 
            df[f'{t}-{i}'] = df[i].shift(t)

The next plot show the amount of Nans for each column

In [ ]:
df.isna().sum().plot(kind='bar', figsize = (12, 1));

In [ ]:
df = df.backfill() # somehow fillna('backfill') produced columns with objects.')
df.dropna(inplace = True) # those rows which contain NaNs due to the shift-method are now removed

As visible in the upcoming slide, Nans are not present any more

In [ ]:
df.isna().sum().plot(kind='bar', figsize = (12, 1));

In [ ]:
df.set_index('timestamp', inplace =True)
df.index.freq = 'min' # df.index gets the frequency of the time series. It is needed for future steps.

In [ ]:
df_180 = df[df.columns[50::4]]
df_1440 = df[df.columns[51::4]]
df_2880 = df[df.columns[52::4]]
df_5760 = df[df.columns[53::4]]

### 180 Minutes in Advance Prediction

In [ ]:
number = 180
df_number = df_180
# train/test split time series. The dataset is split roughly 50:50 for training and test
train_df = df_number.loc[df.index < "2018-06-09 10:40:00"]
test_df = df_number.loc[df.index >= "2018-06-09 10:40:00"]

# X_train = train_df.drop(['machine_status', 'Operation'], axis = 1)
X_train = train_df.drop([f'{number}-machine_status', f'{number}-Operation'], axis = 1)
y_train = train_df[f'{number}-Operation']

# X_test = test_df.drop(['machine_status', 'Operation'], axis = 1)
X_test = test_df.drop([f'{number}-machine_status', f'{number}-Operation'], axis = 1)
X_test_plot = test_df.drop([f'{number}-machine_status'], axis = 1) # this is needed for the upcoming plot
y_test = test_df[f'{number}-Operation']

In [ ]:
# Time when the machine status is broken
X_test_plot[X_test_plot['180-Operation'] == 0].index.tolist()

In [ ]:
# Linear Regression 
reg = LinearRegression()
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)

In [ ]:
y_test_plot = y_test.copy()
y_test_plot = pd.DataFrame(y_test_plot)
y_test_plot['y_pred'] = y_pred.tolist()
y_test_plot.plot.line(figsize=(15,2));
plt.title(f'Linear Regression Prediction {number} Minutes in Advance');

In [ ]:
print('RMSE for Linear Regression: ', "%.3f" % mean_squared_error(y_pred, y_test)**(1/2))

In [ ]:
# Random Forest
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

In [ ]:
y_test_plot = y_test.copy()
y_test_plot = pd.DataFrame(y_test_plot)
y_test_plot['y_pred'] = y_pred.tolist()
y_test_plot.plot.line(figsize=(15,2));
plt.title('Random Forest Prediction 180 Minutes in Advance');

### 1440 Minutes in Advance Prediction

In [ ]:
number = 1440
df_number = df_1440
# train/test split time series. The dataset is split roughly 50:50 for training and test
train_df = df_number.loc[df.index < "2018-06-09 10:40:00"]
test_df = df_number.loc[df.index >= "2018-06-09 10:40:00"]

# X_train = train_df.drop(['machine_status', 'Operation'], axis = 1)
X_train = train_df.drop([f'{number}-machine_status', f'{number}-Operation'], axis = 1)
y_train = train_df[f'{number}-Operation']

# X_test = test_df.drop(['machine_status', 'Operation'], axis = 1)
X_test = test_df.drop([f'{number}-machine_status', f'{number}-Operation'], axis = 1)
X_test_plot = test_df.drop([f'{number}-machine_status'], axis = 1) # this is needed for the upcoming plot
y_test = test_df[f'{number}-Operation']

In [ ]:
# Linear Regression 
reg = LinearRegression()
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)

In [ ]:
y_test_plot = y_test.copy()
y_test_plot = pd.DataFrame(y_test_plot)
y_test_plot['y_pred'] = y_pred.tolist()
y_test_plot.plot.line(figsize=(15,2));
plt.title(f'Linear Regression Prediction {number} Minutes in Advance');

In [ ]:
print('RMSE for Linear Regression: ', "%.3f" % mean_squared_error(y_pred, y_test)**(1/2))

In [ ]:
# Random Forest
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

In [ ]:
y_test_plot = y_test.copy()
y_test_plot = pd.DataFrame(y_test_plot)
y_test_plot['y_pred'] = y_pred.tolist()
y_test_plot.plot.line(figsize=(15,2));
plt.title(f'Random Forest Prediction {number} Minutes in Advance');

In [ ]:
print('RMSE for Random Forest: ', "%.3f" % mean_squared_error(y_pred, y_test)**(1/2))

### 2880 Minutes in Advance Prediction

In [ ]:
number = 2880
df_number = df_2880
# train/test split time series. The dataset is split roughly 50:50 for training and test
train_df = df_number.loc[df.index < "2018-06-09 10:40:00"]
test_df = df_number.loc[df.index >= "2018-06-09 10:40:00"]

# X_train = train_df.drop(['machine_status', 'Operation'], axis = 1)
X_train = train_df.drop([f'{number}-machine_status', f'{number}-Operation'], axis = 1)
y_train = train_df[f'{number}-Operation']

# X_test = test_df.drop(['machine_status', 'Operation'], axis = 1)
X_test = test_df.drop([f'{number}-machine_status', f'{number}-Operation'], axis = 1)
X_test_plot = test_df.drop([f'{number}-machine_status'], axis = 1) # this is needed for the upcoming plot
y_test = test_df[f'{number}-Operation']

In [ ]:
# Linear Regression 
reg = LinearRegression()
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)

In [ ]:
y_test_plot = y_test.copy()
y_test_plot = pd.DataFrame(y_test_plot)
y_test_plot['y_pred'] = y_pred.tolist()
y_test_plot.plot.line(figsize=(15,2));
plt.title(f'Linear Regression Prediction {number} Minutes in Advance');

In [ ]:
print('RMSE for Linear Regression: ', "%.3f" % mean_squared_error(y_pred, y_test)**(1/2))

In [ ]:
# Random Forest
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

In [ ]:
y_test_plot = y_test.copy()
y_test_plot = pd.DataFrame(y_test_plot)
y_test_plot['y_pred'] = y_pred.tolist()
y_test_plot.plot.line(figsize=(15,2));
plt.title(f'Random Forest Prediction {number} Minutes in Advance');

In [ ]:
print('RMSE for Random Forest: ', "%.3f" % mean_squared_error(y_pred, y_test)**(1/2))

### 5760 Minutes in Advance Prediction

In [ ]:
number = 5760
df_number = df_5760
# train/test split time series. The dataset is split roughly 50:50 for training and test
train_df = df_number.loc[df.index < "2018-06-09 10:40:00"]
test_df = df_number.loc[df.index >= "2018-06-09 10:40:00"]

# X_train = train_df.drop(['machine_status', 'Operation'], axis = 1)
X_train = train_df.drop([f'{number}-machine_status', f'{number}-Operation'], axis = 1)
y_train = train_df[f'{number}-Operation']

# X_test = test_df.drop(['machine_status', 'Operation'], axis = 1)
X_test = test_df.drop([f'{number}-machine_status', f'{number}-Operation'], axis = 1)
X_test_plot = test_df.drop([f'{number}-machine_status'], axis = 1) # this is needed for the upcoming plot
y_test = test_df[f'{number}-Operation']

In [ ]:
# Linear Regression 
reg = LinearRegression()
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)

In [ ]:
y_test_plot = y_test.copy()
y_test_plot = pd.DataFrame(y_test_plot)
y_test_plot['y_pred'] = y_pred.tolist()
y_test_plot.plot.line(figsize=(15,2));
plt.title(f'Linear Regression Prediction {number} Minutes in Advance');

In [ ]:
y_test_plot = y_test.copy()
y_test_plot = pd.DataFrame(y_test_plot)
y_test_plot['y_pred'] = y_pred.tolist()
y_test_plot.plot.line(figsize=(15,2));
plt.title(f'Random Forest Prediction {number} Minutes in Advance');

In [ ]:
print('RMSE for Random Forest: ', "%.3f" % mean_squared_error(y_pred, y_test)**(1/2))